In [ ]:
'''
pip install -U FlagEmbedding
pip install -U langchain-community
pip install -v gptqmodel --no-build-isolation
pip install --upgrade optimum transformers
pip install faiss-cpu
'''

'\npip install -U FlagEmbedding\npip install -U langchain-community\npip install -v gptqmodel --no-build-isolation\npip install --upgrade optimum transformers\npip install faiss-cpu\n'

In [89]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
from langchain.vectorstores import FAISS
from FlagEmbedding import FlagAutoModel

embed_model = FlagAutoModel.from_finetuned('BAAI/bge-base-en-v1.5',
                                      query_instruction_for_retrieval="Represent this sentence for searching relevant passages:",
                                      use_fp16=True)

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig, pipeline

model_name = "deepseek-ai/deepseek-llm-7b-chat"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16, device_map="auto")
model.generation_config = GenerationConfig.from_pretrained(model_name)
model.generation_config.pad_token_id = model.generation_config.eos_token_id


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_model-00002-of-00002.bin:  21%|##1       | 818M/3.85G [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   8%|8         | 807M/9.97G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [13]:
from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFacePipeline

In [6]:
vector_store_path = "/content/drive/MyDrive/my_vector_store"
vector_store = FAISS.load_local(vector_store_path, embeddings=embed_model, allow_dangerous_deserialization=True)

In [104]:
torch.cuda.empty_cache()
query = "Side effects of ACETAMINOPHEN"
embed_query = embed_model.encode([query])

In [105]:
if "pregnant" in query.lower():
    category = 'specific population usage'

elif "uses" in query.lower():
    category = 'general'

else:
    category = 'general'

In [106]:
results = vector_store.similarity_search_by_vector(
    embed_query[0],
    k=1,
    filter={"category": category}
)

cleaned_context = [doc.page_content for doc in results]
cleaned_context

['Summary of ACETAMINOPHEN | Uses: Uses • temporarily relieves minor aches and pains due to: • headache • muscular aches • backache • toothache • minor pain of arthritis • the common cold • premenstrual and menstrual cramps • temporarily reduces fever and  |  and  | Reactions: ']

In [107]:
prompt_template = """
You are an expert medical assistant explaining medicine to someone without any prior knowledge.

Use the provided context to answer the user's question ******. If the context does not contain the answer, say "The context does not provide enough information."

# Context: {context}

# Question: {question}

Answer:
"""

In [108]:
# Look at token generation

prompt = PromptTemplate(template=prompt_template, input_variables=['context', 'question'])
llm = HuggingFacePipeline(
    pipeline=pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=300)
)

Device set to use cuda:0


In [109]:
qa_chain = prompt | llm

In [110]:
response = qa_chain.invoke({'context' : cleaned_context, 'question' : query})

def extract_answer(text):
    # Ensure "Answer: " exists in the string
    if "Answer:" in text:
        return text.split("Answer:", 1)[1]

result = extract_answer(response)
result

'\nThe side effects of ACETAMINOPHEN can include stomach upset, dizziness, and skin rash. These side effects are generally mild and go away on their own, but if they persist or worsen, you should consult a doctor. Additionally, ACETAMINOPHEN can cause more serious side effects such as allergic reactions, skin reactions, and bleeding problems. It is important to inform your doctor about any medical conditions you have, as well as any medications you are taking, before using ACETAMINOPHEN, as it may interact with these medications.'


# Example List:

- ## BENZOCAINE
- ## NITROGLYCERIN
- ## ETHANOL
- ## HYDROCHLOROTHIAZIDE
- ## ACETAMINOPHEN
- ## MENTHOL
- ## HYDROCORTISONE
- ## NIACINAMIDE